In [1]:
import os
import datetime
import math
import pandas as pd
from collections import Counter
import helper as hp
from helper import Symbol, Box, GFX, Font, Color, Pin, Property, PinName, mils_to_units, units_to_mils, side_dict, side_idx2val

In [2]:
out_dir = 'Created'
symbol_name = 'test_symbol'

# Defaults
symbol_type = 'Module'

In [3]:
# f = open(os.path.join(out_dir, symbol_name) + '.1', 'w')

In [4]:
def write_str_list(f, str_list):
    for s in str_list:
        f.write(s)
        f.write('\n') 

In [5]:
"""
Estimate the box outline width
lbl_list: list of all pin labels
lbl_size: label size in mils
"""
def estimate_width(lbl_list, lbl_size=100):
    k = 22/38
    max_w = max([len(l) for l in lbl_list])
    return math.ceil((int((max_w*k + 1)*2))/3)*3 * 100

def estimate_height(count):
    return (math.ceil(count/2) + 2) * 100

In [6]:
# def symbol_from_count(symbol_name, count, pin_len = 300):
#     f = open(os.path.join(out_dir, symbol_name) + '.1', 'w')
#     create_main_hdr(f)
    
#     w = estimate_width([str(i) for i in range(1, count + 1)])
#     h = estimate_height(count)
#     print('{}x{}'.format(w, h))
    
#     x = 0
#     y = 0
#     side = 'Left'
#     t_dict = {'Left': 'IN', 'Right': 'OUT'}
#     for i in range(1, count + 1):
#         add_pin(f, i, x, y, side, False, i, str(i), t_dict[side])
#         y += 100
#         if i >= count/2 and side=='Left':
#             side = 'Right'
#             y = 0
#             x += w + 2*pin_len
            
#     add_box(f, 300, -100, w, h)    
#     f.write('E')
#     f.close()

In [7]:
# symbol_from_count(symbol_name, 21)

# Create symbol from CSV

In [8]:
csv_file = 'import.csv'
csv = pd.read_csv(csv_file, delimiter=';')

In [9]:
def get_diff_pairs(df):
    lbl_list = list(df['Pin Label'].values)
    diff_list = [l[:-1] if l.endswith(('_P', '_N')) else l for l in lbl_list]
    cnt = Counter(diff_list)
    diff_list = [l for l, v in cnt.items() if v > 1] # list labels that have both an _N and _P counterpart
    diff_list_full = [l for l in lbl_list if l.endswith(('_P', '_N')) and l[:-1] in diff_list] # get original label names for above list
    
    df['Diff'] = [diff_list.index(l[:-1]) if l in diff_list_full else -1 for l in lbl_list]
    return df

# TODO: SPI, I2C, etc.
def get_bus_signals(df):
    pass

In [10]:
"""
Pin order is defined as 0 to n from bottom to top
df [DataFrame]: full csv as dataframe
"""
def sort_pin_df(df):
    sort = ['Side', 'PWR', 'NC', 'PAD', 'GND', 'Diff', 'INV', 'Pin Number']
    
    nc = []
    pad = []
    gnd = []
    pwr = []
    inv = []
    
    idx = {col:i+1 for i, col in enumerate(df.columns)}
    for r in df.itertuples():
        i = r[0] # dataframe index
        lbl = r[idx['Pin Label']]
        num = r[idx['Pin Number']]
        ptype = r[idx['Pin Type']]
        side = r[idx['Side']]
        inverted = r[idx['Inverted']]
        diff = r[idx['Diff']]
        
        # Pins on symbols bottom should be lower numbers: 0
        # Pin at top should be higher numbers: n
        nc += [0 if lbl.startswith('NC') else 1]
        pad += [0 if 'PAD' in lbl else 1]
        gnd += [0 if ptype == 'GROUND' else 1]
        pwr += [1 if ptype == 'POWER' else 0] 
        inv += [0 if inverted else 1]
    
    df.loc[:, 'NC'] = nc
    df.loc[:, 'PAD'] = pad
    df.loc[:, 'GND'] = gnd
    df.loc[:, 'PWR'] = pwr
    df.loc[:, 'INV'] = inv
    
    df.sort_values(by=sort, inplace=True)
    df.reset_index(inplace=True, drop=True)
    
    df.loc[df.Side == side_dict['Left'], 'sort'] = [i for i in range(len(df.loc[df.Side == side_dict['Left'], 'Side']))]
    df.loc[df.Side == side_dict['Right'], 'sort'] = [i for i in range(len(df.loc[df.Side == side_dict['Right'], 'Side']))]
    
    return df

In [11]:
import numpy as np

def get_gaps(df):
    gap_list = []
    prev_diff = False
    gaps = np.zeros((len(df)))
    
    idx = {col:i+1 for i, col in enumerate(df.columns)}
    for i, r in enumerate(df.itertuples()):
        
        lbl = r[idx['Pin Label']]
        ptype = r[idx['Pin Type']]
        side = r[idx['Side']]
        inv = r[idx['Inverted']]
        diff = r[idx['Diff']]
        sort = r[idx['sort']]

        # Add gaps to differential pairs
        if diff and inv:
            gaps[i] = 1
        elif diff and not(inv):
            if i + 1 < len(df):
                gaps[i+1] = 2
            
        if lbl.startswith('NC'):
            if i + 1 < len(df):
                gaps[i+1] = 1
            
        if gaps[i] == 2:
            if not(diff):
                gaps[i] = 1
        
        # Don't add gaps if this is the first pin
        if sort == 0:
            gaps[i] = 0
        
    df['gap'] = gaps
    
    return df

In [12]:
def get_coordinates(df, w, pin_len=300):
    df = sort_pin_df(df)
    df = get_gaps(df)
    
    x = []
    y = []
    
    idx = {col:i+1 for i, col in enumerate(df.columns)}
    for r in df.itertuples():
        i = r[0] # dataframe index
        order = r[idx['Pin Order']]
        lbl = r[idx['Pin Label']]
        num = r[idx['Pin Number']]
        ptype = r[idx['Pin Type']]
        side = r[idx['Side']]
        inv = r[idx['Inverted']]
        diff = r[idx['Diff']]
        sort = r[idx['sort']]
        gap = r[idx['gap']]
        
        if sort == 0:
            gaps = 0
        
        gaps += gap*100
        x += [0 if side == side_dict['Left'] else w + pin_len*2]
        y += [sort*100 + gaps]
        
    df.loc[:, 'x'] = x
    df.loc[:, 'y'] = y
    
    return df

In [13]:
def symbol_from_csv(csv_file, symbol_name, pin_len = 300, box_margin=100):    
    # Import CSV
    df = pd.read_csv(csv_file, delimiter=';')
    df = get_diff_pairs(df)
    
    w = estimate_width(list(df['Pin Label'].values))
    
    # Get Coordinates
    df = get_coordinates(df, w, pin_len=pin_len)
    
    h = max(df.y.values) + box_margin*2
    print('{}x{}'.format(w, h))
    
    symbol = Symbol()
    idx = {col:i+1 for i, col in enumerate(df.columns)}
    for r in df.itertuples():
        i = r[0] # dataframe index
        order = r[idx['Pin Order']]
        lbl = r[idx['Pin Label']]
        num = r[idx['Pin Number']]
        ptype = r[idx['Pin Type']]
        side = r[idx['Side']]
        inv = r[idx['Inverted']]
        diff = r[idx['Diff']]
        x = r[idx['x']]
        y = r[idx['y']]
        
        symbol.add_pin(i+1, x, y, side_idx2val[side], inv, num, lbl, ptype)
    
    box = symbol.add_box(300, -100, w, h)
    
    f = open(os.path.join(out_dir, symbol_name) + '.1', 'w')
    write_str_list(f, symbol.get_symbol_str_list())
    f.close()

In [14]:
symbol_from_csv(csv_file, symbol_name)

1200x2900.0


In [15]:
w = estimate_width(list(csv['Pin Label'].values))
csv = get_diff_pairs(csv)
get_coordinates(csv, w)

,Pin Order,Pin Label,Pin Number,Pin Type,Side,Inverted,Diff,NC,PAD,GND,PWR,INV,sort,gap,x,y
0,L8,SEN_N,7,IN,2,True,-1,1,1,1,0,0,0.0,0.0,0,0.0
1,L13,SYNC_N,63,IN,2,True,-1,1,1,1,0,0,1.0,1.0,0,200.0
2,L7,SDIN,5,IN,2,False,-1,1,1,1,0,1,2.0,0.0,0,300.0
3,L6,SCLK,6,IN,2,False,-1,1,1,1,0,1,3.0,2.0,0,600.0
4,L9,VCM,22,ANALOG,2,False,-1,1,1,1,0,1,4.0,2.0,0,900.0
5,L5,RESET,48,IN,2,False,-1,1,1,1,0,1,5.0,2.0,0,1200.0
6,L12,PDN/OVR,50,BI,2,False,-1,1,1,1,0,1,6.0,2.0,0,1500.0
7,L1,CLK_N,28,IN,2,True,0,1,1,1,0,0,7.0,1.0,0,1700.0
8,L2,CLK_P,27,IN,2,False,0,1,1,1,0,1,8.0,0.0,0,1800.0
9,L3,INA_N,41,ANALOG,2,True,8,1,1,1,0,0,9.0,1.0,0,2000.0
